### This notebook is dedicated to describing experimental data collected in the following publication:

Cai, H., & Heer, P. (2024). Experimental implementation of an emission-aware prosumer with online flexibility quantification and provision. Sustainable Cities and Society, 101, 105531.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import requests

#### Download data from the shared folder.

In [ ]:
# Data are placed in folder ./data/
# For manual data download, please use: https://polybox.ethz.ch/index.php/s/Ti8FyQ38diDC7SD
# The following code download automatically with URLs of the data files in different format. Choose as you prefer.
urls = [
    'https://polybox.ethz.ch/index.php/s/JjiqQ3rVLDS2LJ8/download',
    # 'https://polybox.ethz.ch/index.php/s/ZMyaE1DWR5ngQ0R/download',
    # 'https://polybox.ethz.ch/index.php/s/k3irx1SEw1QNkDt/download'
]

# Corresponding filenames
filenames = [
    'experimental_results.csv',
    # 'experimental_results.pkl',
    # 'experimental_results.h5'
]

# Directory to save the downloaded files
data_folder = 'data'

# Create the data folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

for url, filename in zip(urls, filenames):
    response = requests.get(url)
    file_path = os.path.join(data_folder, filename)

    # Ensure the request was successful
    if response.status_code == 200:
        # Save the file as is
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded and saved as {file_path}")

    else:
        print(f"Failed to download file from {url}. Status code: {response.status_code}")

print("Download process completed.")

In [ ]:
# Note that Zurich time (CET) was used

# Read the dataframe from csv format
df = pd.read_csv('data/experimental_results.csv', encoding='ISO-8859-1')
df.set_index('timestamp', inplace=True)

# Read the dataframe from pickle format
# df = pd.read_pickle('data/experimental_results.pkl')
    
# Read the dataframe from HDF5 format
# df = pd.read_hdf('data/experimental_results.h5', key='df')

# # Ensure the index is a DatetimeIndex
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

### Space heating

In [ ]:
def plot_sh_result(df_sub):

    plt.figure(figsize=(7, 14))
    plt.subplot(5, 1, 1)
    
    # Plot temperature within each room/zone
    plt.plot(df_sub.index, df_sub['temp_272'], label='Room 272')
    plt.plot(df_sub.index, df_sub['temp_273'], label='Room 273')
    plt.plot(df_sub.index, df_sub['temp_274'], label='Room 274')
    plt.plot(df_sub.index, df_sub['upper_lim'], 'k--')
    plt.plot(df_sub.index, df_sub['lower_lim'], 'k--')
    plt.ylabel("Temperature [$^o$C]", fontsize=10)
    plt.grid()
    plt.legend(loc=0)
    
    plt.subplot(5, 1, 4)
    # First calculate temperature deviations outside comfort zone
    df_sub['r272_dev'] = df_sub['lower_lim'] - df_sub['temp_272']
    df_sub.loc[df_sub['r272_dev'] < 0, 'r272_dev'] = 0
    df_sub['r273_dev'] = df_sub['lower_lim'] - df_sub['temp_273']
    df_sub.loc[df_sub['r273_dev'] < 0, 'r273_dev'] = 0
    df_sub['r274_dev'] = df_sub['lower_lim'] - df_sub['temp_274']
    df_sub.loc[df_sub['r274_dev'] < 0, 'r274_dev'] = 0
    
    # Then plot accumulated temperature deviation
    plt.plot(df_sub.index, np.cumsum(df_sub['r272_dev'] / 4), label='Room 272')
    plt.plot(df_sub.index, np.cumsum(df_sub['r273_dev'] / 4), label='Room 273')
    plt.plot(df_sub.index, np.cumsum(df_sub['r274_dev'] / 4), label='Room 274')
    plt.ylabel("Accum. temp. violation [Kh]", fontsize=10)
    plt.grid()
    plt.legend(loc=0)    
    
    plt.subplot(5, 1, 5)
    # Plot boundary conditions
    plt.plot(df_sub.index, df_sub['meas_temp'], label='Temperature', color='blue')
    plt.ylabel("Ambient temperature [$^o$C]", fontsize=10)
    plt.legend(loc=2)

    plt2 = plt.twinx()
    plt2.plot(df_sub.index, df_sub['meas_irrd'], label='Irradiance', color='green')
    plt.ylabel("Irradiance [W/m$^2$]", fontsize=10)
    plt.legend(loc=0)
    plt.grid()
    
    # Reduce time resolution for better visualization
    df_sub = df_sub.resample('60T').mean()

    # Plot the measured and anticipated thermal power inputs
    plt.subplot(5, 1, 2)
    plt.step(df_sub.index, df_sub['power_272_h'], label='Room 272 meas.')
    plt.step(df_sub.index, df_sub['power_273_h']/2, label='Room 273 meas.') # the number 2 here refers to power capacity
    plt.step(df_sub.index, df_sub['power_274_h'], label='Room 274 meas.')   
    plt.ylabel("Relative power input [-]", fontsize=10)
    plt.grid()
    plt.legend(loc=2)
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['forecast'], 'k--', label='Emission')
    plt.ylabel("Intensity [gCO$_2$ eq/kWh]", fontsize=10)
    plt.legend(loc=1) 
    
    plt.subplot(5, 1, 3)
    plt.step(df_sub.index, df_sub['power_272'], '--', label='Room 272 ctrl.')
    plt.step(df_sub.index, df_sub['power_273']/2, '--', label='Room 273 ctrl.')  # the number 2 here refers to power capacity
    plt.step(df_sub.index, df_sub['power_274'], '--', label='Room 274 ctrl.')
    plt.ylim(0, 1)
    plt.ylabel("Relative power input [-]", fontsize=10)
    plt.grid()
    plt.legend(loc=2)
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['forecast'], 'k--', label='Emission')
    plt.ylabel("Intensity [gCO$_2$ eq/kWh]", fontsize=10)
    plt.legend(loc=1)
    plt.gcf().autofmt_xdate()  
    plt.tight_layout()
    
plot_sh_result(df)

### Domestic hot water heating

In [ ]:
def plot_dhw_result(df_sub):
   
    plt.figure(figsize=(7, 5))
    plt.subplot(2, 1, 1)

    # Plot average tank temperature and limits
    plt.plot(df_sub.index, df_sub['avg_temp'], label='Tank avg temp', color='blue')
    plt.plot(df_sub.index, df_sub['dhw_upper_lim'], 'k--')
    plt.plot(df_sub.index, df_sub['dhw_lower_lim'], 'k--')
    plt.ylim(40, 65)
    plt.ylabel("Temperature [$^o$C]", fontsize=10)
    df_sub['water_draw'] = df_sub['accum_flow'].shift(-1) - df_sub['accum_flow']
    plt.legend(loc=2)

    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['water_draw'] * 1000 / 15, label='Water draw', color='green')
    plt.ylabel("Water draw [L/min]", fontsize=10)
    plt.ylim(0,6)
    plt.grid()
    plt.legend(loc=0)

    plt.subplot(2, 1, 2)
    # Plot measured and anticipated thermal power into the tank
    plt.plot(df_sub.index, df_sub['charge_power'] / 5.7, label='Measured')
    plt.plot(df_sub.index, df_sub['power_dhw'], label='Control')
    plt.ylabel("Relative power input [-]", fontsize=10)
    plt.grid()
    plt.legend(loc=2)   
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['forecast'], 'k--', label='Emission')
    plt.ylabel("Intensity [gCO$_2$ eq/kWh]", fontsize=10)
    plt.legend(loc=1)
    plt.tight_layout()
    plt.gcf().autofmt_xdate()  # rotate time index
    plt.show()

plot_dhw_result(df)

### Other electric devices

In [ ]:
def plot_nonthermal_result(df_sub):
    df_sub.loc[df_sub['at_home'] == 0, 'power_ev'] = np.nan
    df_sub.loc[df_sub['at_home'] == 0, 'SOC_ini_value_ev'] = np.nan

    plt.figure(figsize=(7, 14))
    plt.subplot(5, 1, 1)
    # Plot data related to battery
    plt.plot(df_sub.index, df_sub['power_ebat'], label='Battery optimized power', linestyle='dashed')
    plt.plot(df_sub.index, df_sub['inverter_reactive_P_AC'], label='Battery measured power')
    plt.ylabel("Power [kW]", fontsize=10)
    plt.legend(loc=2)
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['SOC'], label='SOC', color='green')  # the scaling is due to emulating a battery smaller than what we actually have
    plt.ylabel("SOC [%]", fontsize=10)
    plt.legend(loc=0)
    plt.grid()
    
    plt.subplot(5, 1, 2)
    # Plot data related to EV
    plt.plot(df_sub.index, df_sub['at_home'], 'k--', label='Connected at home')
    plt.plot()
    plt.legend(loc=2)
    plt.ylabel("User pattern", fontsize=10)
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['power_ev'], label='EV optimized power')
    plt2.legend(loc=1)
    plt.ylabel("Power [kW]", fontsize=10)
    plt.grid()
    
    plt.subplot(5, 1, 3)
    plt.plot(df_sub.index, df_sub['at_home'], 'k--', label='Connected at home')
    plt.plot()
    plt.ylabel("User pattern", fontsize=10)
    plt.legend(loc=2)
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['SOC_ini_value_ev'], label='EV SOC') # the scaling is due to emulating a battery smaller than what we actually have
    plt2.legend(loc=1)
    plt.ylabel("SOC [%]", fontsize=10)
    plt.grid()
    
    plt.subplot(5, 1, 4)
    plt.plot(df_sub.index, - df_sub['active_power_total_2'],label='PV')
    plt.plot()
    plt.ylabel("Power [kW]", fontsize=10)
    plt.legend(loc=2)
    plt.grid()
    
    plt.subplot(5, 1, 5)
    plt.plot(df_sub.index, df_sub['fixed_load'],label='Uncontrolled load')
    plt.plot()
    plt.ylabel("Power [kW]", fontsize=10)
    plt.legend(loc=2)
    plt.grid()
    plt.tight_layout()
    plt.gcf().autofmt_xdate()  # rotate time index

plot_nonthermal_result(df)

### Grid exchange point

In [ ]:
def plot_grid_result(df_sub): 

    plt.figure(figsize=(7, 3))
    plt.plot(df_sub.index, df_sub['total_power'], label='Total power')   
    
    plt.ylabel("Power [kW]", fontsize=10)
    plt.legend(loc=2)
    plt2 = plt.twinx()
    plt2.step(df_sub.index, df_sub['forecast'], 'k--', label='Emission')
    plt.ylabel("Intensity [gCO$_2$ eq/kWh]", fontsize=10)
    plt.legend(loc=1)
    plt.grid()
    plt.gcf().autofmt_xdate()  # rotate time index
    plt.show()

plot_grid_result(df)

In [ ]:
used_columns = {
    'temp_272', 'temp_273', 'temp_274', 'upper_lim', 'lower_lim',
    'meas_temp', 'meas_irrd', 'power_272_h', 'power_273_h', 'power_274_h',
    'forecast', 'power_272', 'power_273', 'power_274',
    'avg_temp', 'dhw_upper_lim', 'dhw_lower_lim', 'accum_flow', 'charge_power',
    'power_dhw', 'at_home', 'power_ev', 'SOC_ini_value_ev', 'power_ebat',
    'inverter_reactive_P_AC', 'SOC', 'active_power_total_2', 'fixed_load',
    'power_sp', 'total_power'
}

# List of all columns in the dataframe
all_columns = set(df.columns)

# Identify unused columns
unused_columns = all_columns - used_columns

# Remove unused columns from the dataframe
df = df.drop(columns=unused_columns)

In [ ]:
# df.to_pickle('data/experimental_results.pkl')
# df.to_hdf('data/experimental_results.h5', key='df', mode='w')
# df.to_csv('data/experimental_results.csv')

#### That's it. Let me know if things do not work out.